
# GONI Example

This example demonstrates fitting a penalized spherical spline to
the GONI dataset and plotting the resulting spline curve on a world map.

<img src="file://_static/thumbnails/goni_thumb.png" class="sphx-glr-thumbimg">


In [ ]:
# ----------------------------------------------------
# Imports
# ----------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

import spheresmooth as ss
from spheresmooth.loads import load_world_map


# ----------------------------------------------------
# 1. Load GONI spherical data (t, theta, phi)
# ----------------------------------------------------
goni = ss.load_goni()   # DataFrame: [t, theta, phi]

t = goni.iloc[:, 0].values
spherical = goni.iloc[:, 1:3].values   # (theta, phi)


# ----------------------------------------------------
# 2. Spherical → Cartesian
# ----------------------------------------------------
goni_cartesian = ss.spherical_to_cartesian(spherical)


# ----------------------------------------------------
# 3. Quantile knots + lambda sequence
# ----------------------------------------------------
dimension = 12     # moderate for quick Sphinx-Gallery execution
initial_knots = ss.knots_quantile(t, dimension)

lambda_seq = np.exp(np.linspace(np.log(1e-6), np.log(1), 25))


# ----------------------------------------------------
# 4. Penalized spherical spline fit
# ----------------------------------------------------
fit = ss.penalized_linear_spherical_spline(
    t=t,
    y=goni_cartesian,
    dimension=dimension,
    initial_knots=initial_knots,
    lambdas=lambda_seq
)

bic_list = fit["bic_list"]
fits = fit["fits"]

best_index = np.argmin(bic_list)
best_fit = fits[best_index]
control_points = best_fit["control_points"]
knots = best_fit["knots"]

print("Best λ index:", best_index)
print("Control points (cartesian):")
print(control_points)


# ----------------------------------------------------
# 5. Convert control points to latitude/longitude
# ----------------------------------------------------
cp_spherical = ss.cartesian_to_spherical(control_points)
cp_deg = np.degrees(cp_spherical)

cp_df = pd.DataFrame({
    "latitude": 90 - cp_deg[:, 0],
    "longitude": cp_deg[:, 1],
})


# ----------------------------------------------------
# 6. Evaluate piecewise geodesic curve
# ----------------------------------------------------
ts = np.linspace(0, 1, 2000)

curve_cart = ss.piecewise_geodesic(ts, control_points, knots)
curve_sph = ss.cartesian_to_spherical(curve_cart)
curve_deg = np.degrees(curve_sph)

curve_df = pd.DataFrame({
    "latitude": 90 - curve_deg[:, 0],
    "longitude": curve_deg[:, 1],
})


# ----------------------------------------------------
# 7. Original GONI data → (lat, lon)
# ----------------------------------------------------
goni_deg = np.degrees(spherical)

goni_df = pd.DataFrame({
    "latitude": 90 - goni_deg[:, 0],
    "longitude": goni_deg[:, 1],
})


# # ----------------------------------------------------
# # 8. Figure 1: World map + GONI + fitted curve
# # ----------------------------------------------------
# fig, ax = plt.subplots(figsize=(14, 6))

# # Load world shapefile
# world_path = load_world_map()
# world = gpd.read_file(world_path)

# # World map
# world.plot(ax=ax, color="antiquewhite", edgecolor="gray")

# # Raw GONI data
# ax.scatter(
#     goni_df["longitude"], goni_df["latitude"],
#     s=3, color="black", label="GONI data"
# )

# # Control points
# ax.scatter(
#     cp_df["longitude"], cp_df["latitude"],
#     s=50, color="blue", marker="s", label="Control points"
# )

# # Fitted spline curve
# ax.plot(
#     curve_df["longitude"], curve_df["latitude"],
#     color="red", linewidth=1.5, label="Spline curve"
# )

# ax.set_xlabel("longitude")
# ax.set_ylabel("latitude")
# ax.set_title("GONI Data + Control Points + Fitted Spherical Spline")
# ax.legend()

# plt.show()  



# ----------------------------------------------------
# 9. Figure 2: Zoom-in (East Asia)
# ----------------------------------------------------
fig2, ax2 = plt.subplots(figsize=(14, 10))

# World map
world.plot(ax=ax2, color="antiquewhite", edgecolor="gray")

# Raw GONI data
ax2.scatter(
    goni_df["longitude"], goni_df["latitude"],
    s=5, color="black", label="GONI data"
)

# Control points
ax2.scatter(
    cp_df["longitude"], cp_df["latitude"],
    s=60, color="blue", marker="s", label="Control points"
)

# Fitted spline curve
ax2.plot(
    curve_df["longitude"], curve_df["latitude"],
    color="red", linewidth=2, label="Spline curve"
)

# Zoom bounds (East Asia)
ax2.set_xlim(100, 170)
ax2.set_ylim(-10, 65)

ax2.set_xlabel("longitude")
ax2.set_ylabel("latitude")
ax2.set_title("Zoomed-in Region: East Asia")
ax2.legend()

plt.show()